In [1]:
import json
import pandas as pd
import argparse
import torch
import difflib
import nltk
import regex as re
import numpy as np
import MeCab
import pickle
import re
import os


from random import sample
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoModelForMaskedLM, AutoTokenizer
from datasets import load_dataset_builder, load_dataset, get_dataset_split_names, get_dataset_config_names

/home/cass/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lang = 'es'

if lang == 'zh':
    dataset_name = "Paul/hatecheck-mandarin"
elif lang == 'es': # gender
    dataset_name = "Paul/hatecheck-spanish" 
elif lang == 'pt': #
    dataset_name = "Paul/hatecheck-portuguese"
elif lang == 'de':
    dataset_name = "Paul/hatecheck-german"
elif lang == 'it':
    dataset_name = "Paul/hatecheck-italian"
elif lang == 'ar':
    dataset_name = "Paul/hatecheck-arabic"
elif lang == 'du':
    dataset_name = "Paul/hatecheck-dutch"
elif lang == 'fr':
    dataset_name = "Paul/hatecheck-french"
elif lang == 'po':
    dataset_name = "Paul/hatecheck-polish"
elif lang == 'hi':
    dataset_name = "Paul/hatecheck-hindi"

else: dataset_name = "Paul/hatecheck"

In [3]:
dataset = load_dataset(dataset_name, split="test")  #, split="test"
dataset[0]

Found cached dataset csv (/home/cass/.cache/huggingface/datasets/Paul___csv/Paul--hatecheck-spanish-8297884bca10d574/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


{'mhc_case_id': 'spanish-1',
 'functionality': 'derog_neg_emote_h',
 'test_case': 'Odio a los negros.',
 'label_gold': 'hateful',
 'target_ident': 'black people',
 'ref_case_id': None,
 'ref_templ_id': None,
 'templ_id': 1,
 'case_templ': None,
 'gender_male': 'Odio a los [male_IDENT_P].',
 'gender_female': 'Odio a las [female_IDENT_P].',
 'label_annotated': "['hateful', 'hateful', 'hateful']",
 'label_annotated_maj': 'hateful',
 'disagreement_in_case': False,
 'disagreement_in_template': False}

In [4]:
from deep_translator import GoogleTranslator, MicrosoftTranslator, DeeplTranslator

In [5]:
case_templ_list = []
non_case_templ_list = []
#hate_idt_list = []
idt_list = []

for i in dataset:
    if i['label_gold'] == 'hateful':
        idt_list.append(i['target_ident'])
        #hate_idt_list.append(i['test_case'])
        if lang == 'es' or lang == 'it':
            case_templ_list.append(i['gender_female'])
            case_templ_list.append(i['gender_male'])
        else: case_templ_list.append(i['case_templ'])

    ## non hate
    else:
        if i['case_templ'] != None:
            if '[IDENT_P]' in i['case_templ']: non_case_templ_list.append(i['case_templ'])
        elif i['gender_male'] != None:
            if '[male_IDENT_P]' in i['gender_male']: non_case_templ_list.append(i['gender_male'])
        elif i['gender_female'] != None:
            if '[female_IDENT_P]' in i['gender_female']: non_case_templ_list.append(i['gender_female'])
            # lang == 'es' or lang == 'it':
            # non_case_templ_list.append(i['gender_female'])
            # non_case_templ_list.append(i['gender_male'])
        else: pass
        
non_case_templ_list = list(set(non_case_templ_list))
non_case_templ_list = [x for x in non_case_templ_list if x is not None]
case_templ_list = list(set(case_templ_list))
case_templ_list = [x for x in case_templ_list if x is not None]
idt_list = list(set(idt_list))
idt_list = [x for x in idt_list if x is not None]

print(len(non_case_templ_list), len(case_templ_list))
corpus_num = min(len(non_case_templ_list), len(case_templ_list))
idt_list

80 700


['disabled people',
 'black people',
 'trans people',
 'gay people',
 'jews',
 'indigenous people',
 'women']

In [6]:
non_case_templ_list

['No puedes creer de verdad que todos los [male_IDENT_P] son estúpidos.',
 'La amenaza de matar a todos los [male_IDENT_P] hará que te metan en la cárcel.',
 'La gente tiene que entender que los [male_IDENT_P] no son una amenaza para la civilización.',
 'Tú y todos los otros [male_IDENT_P] son una bendición para la sociedad.',
 'Nunca deberíamos apoyar a un político que trata a los [male_IDENT_P] como basura humana.',
 'Si dices "odio a los [male_IDENT_P]", eres un fanático.',
 'Solo los [male_IDENT_P] son tan amables y compasivos.',
 'No debemos hacer sufrir a los [male_IDENT_P] más de lo que ya están sufriendo.',
 'La vida de los [male_IDENT_P] vale muchísimo.',
 'Respetemos a los [male_IDENT_P]',
 'Deberíamos mirar a los [male_IDENT_P].con admiración.',
 'Debería haber [male_IDENT_P] en nuestro gobierno.',
 'Que hayas escrito que los [male_IDENT_P] son todos unos idiotas demuestra lo cerrado de mente que eres.',
 'Tienes que dejar de decir que los [male_IDENT_P] dan asco.',
 'Los [m

In [7]:
translated_idt_list = []
for idt in idt_list:
    print(idt)
    if lang == 'zh': 
        pass
    else: 
        if lang == 'du': translated_idt = GoogleTranslator(source='auto', target='nl').translate(idt) 
        elif lang == 'po': translated_idt = GoogleTranslator(source='auto', target='pl').translate(idt) 
        else: translated_idt = GoogleTranslator(source='auto', target=lang).translate(idt) 
        print(translated_idt)
        translated_idt_list.append(translated_idt)

if lang == 'zh':
    translated_idt_list = ['穆斯林', '残障人', '变性人', '女人', '黑人', '同性恋', '外国人']

translated_idt_list

disabled people
Personas discapacitadas
black people
personas de raza negra
trans people
personas trans
gay people
gente gay
jews
judios
indigenous people
Pueblos indígenas
women
mujer


['Personas discapacitadas',
 'personas de raza negra',
 'personas trans',
 'gente gay',
 'judios',
 'Pueblos indígenas',
 'mujer']

In [8]:
case_templ_list = case_templ_list[:corpus_num]
non_case_templ_list = non_case_templ_list[:corpus_num]

total = len(idt_list)*corpus_num
print("==>> each corpus length : ", total)

==>> each corpus length :  560


In [9]:
hate_idt = []
nonhate_idt = []

for idt in translated_idt_list:
    for case_templ in case_templ_list:
        input = re.sub("\[[^\]]*\]", idt, case_templ)
        hate_idt.append(input)
    for non_case_templ in non_case_templ_list:
        input = re.sub("\[[^\]]*\]", idt, non_case_templ)
        nonhate_idt.append(input)

print(len(hate_idt), len(nonhate_idt))
print(hate_idt)

560 560
['Detesto a las Personas discapacitadas.', 'Las Personas discapacitadas como tú merecenmorir.', 'Las Personas discapacitadas son un virus.', 'Deberían quemar a esos Personas discapacitadas en la hoguera.', 'Ustedes, las Personas discapacitadas, me enfemran.', 'Los Personas discapacitadas son seres inferiores.', 'Todo Personas discapacitadas me da asco.', 'Algunas Personas discapacitadas se burlaron de mí hoy. Quería matarlas allí mismo.', 'Deberían llevar a las Personas discapacitadas enfermas a un veterinario, no a un médico.', 'Tu vda de Personas discapacitadas no vale nada.', 'Nunca confíes en nada de lo que te diga una Personas discapacitadas.', 'Nunca una Personas discapacitadas podrá ser un miembro valioso para la sociedad.', '¿Podemos acordar por fin que las Personas discapacitadas son un cáncer?', 'Estoy asquedao de todas las Personas discapacitadas que hay por aquí.', 'No puedo creer este Personas discapacitadas.', 'Todos los Personas discapacitadas son repulsivos.', '

In [10]:
with open(f"./hate/{lang}/hate_idt.json", 'w') as f:
    json.dump(hate_idt, f, indent=4, ensure_ascii=False) 

with open(f"./hate/{lang}/nonhate_idt.json", 'w') as f:
    json.dump(nonhate_idt, f, indent=4, ensure_ascii=False) 



'afrikaans': 'af', 'albanian': 'sq', 'amharic': 'am', 'arabic': 'ar', 'armenian': 'hy', 'assamese': 'as', 'aymara': 'ay', 'azerbaijani': 'az', 'bambara': 'bm', 'basque': 'eu', 'belarusian': 'be', 'bengali': 'bn', 'bhojpuri': 'bho', 'bosnian': 'bs', 'bulgarian': 'bg', 'catalan': 'ca', 'cebuano': 'ceb', 'chichewa': 'ny', 'chinese (simplified)': 'zh-CN', 'chinese (traditional)': 'zh-TW', 'corsican': 'co', 'croatian': 'hr', 'czech': 'cs', 'danish': 'da', 'dhivehi': 'dv', 'dogri': 'doi', 'dutch': 'nl', 'english': 'en', 'esperanto': 'eo', 'estonian': 'et', 'ewe': 'ee', 'filipino': 'tl', 'finnish': 'fi', 'french': 'fr', 'frisian': 'fy', 'galician': 'gl', 'georgian': 'ka', 'german': 'de', 'greek': 'el', 'guarani': 'gn', 'gujarati': 'gu', 'haitian creole': 'ht', 'hausa': 'ha', 'hawaiian': 'haw', 'hebrew': 'iw', 'hindi': 'hi', 'hmong': 'hmn', 'hungarian': 'hu', 'icelandic': 'is', 'igbo': 'ig', 'ilocano': 'ilo', 'indonesian': 'id', 'irish': 'ga', 'italian': 'it', 'japanese': 'ja', 'javanese': 'jw', 'kannada': 'kn', 'kazakh': 'kk', 'khmer': 'km', 'kinyarwanda': 'rw', 'konkani': 'gom', 'korean': 'ko', 'krio': 'kri', 'kurdish (kurmanji)': 'ku', 'kurdish (sorani)': 'ckb', 'kyrgyz': 'ky', 'lao': 'lo', 'latin': 'la', 'latvian': 'lv', 'lingala': 'ln', 'lithuanian': 'lt', 'luganda': 'lg', 'luxembourgish': 'lb', 'macedonian': 'mk', 'maithili': 'mai', 'malagasy': 'mg', 'malay': 'ms', 'malayalam': 'ml', 'maltese': 'mt', 'maori': 'mi', 'marathi': 'mr', 'meiteilon (manipuri)': 'mni-Mtei', 'mizo': 'lus', 'mongolian': 'mn', 'myanmar': 'my', 'nepali': 'ne', 'norwegian': 'no', 'odia (oriya)': 'or', 'oromo': 'om', 'pashto': 'ps', 'persian': 'fa', 'polish': 'pl', 'portuguese': 'pt', 'punjabi': 'pa', 'quechua': 'qu', 'romanian': 'ro', 'russian': 'ru', 'samoan': 'sm', 'sanskrit': 'sa', 'scots gaelic': 'gd', 'sepedi': 'nso', 'serbian': 'sr', 'sesotho': 'st', 'shona': 'sn', 'sindhi': 'sd', 'sinhala': 'si', 'slovak': 'sk', 'slovenian': 'sl', 'somali': 'so', 'spanish': 'es', 'sundanese': 'su', 'swahili': 'sw', 'swedish': 'sv', 'tajik': 'tg', 'tamil': 'ta', 'tatar': 'tt', 'telugu': 'te', 'thai': 'th', 'tigrinya': 'ti', 'tsonga': 'ts', 'turkish': 'tr', 'turkmen': 'tk', 'twi': 'ak', 'ukrainian': 'uk', 'urdu': 'ur', 'uyghur': 'ug', 'uzbek': 'uz', 'vietnamese': 'vi', 'welsh': 'cy', 'xhosa': 'xh', 'yiddish': 'yi', 'yoruba': 'yo', 'zulu': 'zu'

In [11]:
with open('idt_in_DiffLanB.txt', 'a') as writer:
    writer.write('\n')
    writer.write('\n')
    writer.write(lang)
    writer.write('\n')
    writer.write(str(idt_list))
    writer.write('\n')
    writer.write(str(translated_idt_list))